In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

与えられた売上データを分析することで、以下のような傾向やインサイトが得られる可能性があります。

### 1. カテゴリー別の売上傾向
- **カテゴリー別売上**: 食品、衣服、日用品のカテゴリーに分かれているため、各カテゴリーの売上総額や販売数量を集計し、どのカテゴリーが最も売上を上げているかを特定できます。これにより、顧客の好みや需要に基づいて在庫や販売戦略を調整できます。

### 2. 商品別のパフォーマンス
- **人気商品**: 商品ごとの売上を計算し、最も売上が高い商品や最も売れた商品のランキングを作成できます。これにより、ヒット商品のプロモーションを強化したり、類似商品の開発を検討するための情報を得ることができます。

### 3. 売上の季節性
- **売上日分析**: 日付ごとの売上データを元に、時系列で分析を行うことで売上の季節性を確認できます。例えば、特定の季節や月に売上が増加する商品があるかもしれません。これを把握することで、販促活動のタイミングを最適化できます。

### 4. 原価と利益分析
- **原価と利益**: 売上、単価、原価を元に各商品の利益を計算し、どの商品の利益率が高いかを評価します。高利益商品に重点を置くことで、利益の最大化を図ることができます。

### 5. 売上のトレンド
- **トレンド分析**: 時間の経過とともに売上が増加しているのか、減少しているのかを確認します。顧客の関心の変化が売上にどのように影響しているかを分析することで、今後の戦略を立てることができます。

### 6. クロスセリングの機会
- **クロスセリングの可能性**: 商品の売上データを組み合わせて、どの組み合わせで購買されているのかを分析し、セット販売やプロモーションの提案を行うことができます。

### 7. 地域別売上（拡張可能性）
- 地域別に売上データがあれば、地域別の傾向を分析し、特定の地域でのプロモーション戦略を適用することができます。

### 結論
これらの傾向を基に戦略的な意思決定を行うことが重要です。データの視覚化や詳細な統計解析を実施することで、より深い洞察を得ることができるでしょう。また、顧客の声やフィードバックも取り入れ、これをデータ分析に活用することで、顧客満足度を向上させることが可能です。


In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0     与えられた売上データを分析することで、以下のような傾向やインサイトが得られる可能性があります。
1                                                    
2                                  ### 1. カテゴリー別の売上傾向
3   - **カテゴリー別売上**: 食品、衣服、日用品のカテゴリーに分かれているため、各カテゴリ...
4                                                    
5                                  ### 2. 商品別のパフォーマンス
6   - **人気商品**: 商品ごとの売上を計算し、最も売上が高い商品や最も売れた商品のランキン...
7                                                    
8                                       ### 3. 売上の季節性
9   - **売上日分析**: 日付ごとの売上データを元に、時系列で分析を行うことで売上の季節性を...
10                                                   
11                                     ### 4. 原価と利益分析
12  - **原価と利益**: 売上、単価、原価を元に各商品の利益を計算し、どの商品の利益率が高い...
13                                                   
14                                     ### 5. 売上のトレンド
15  - **トレンド分析**: 時間の経過とともに売上が増加しているのか、減少しているのかを確認...
16                                                   
17                          

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [7]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
